In [2]:
import os
import motor.motor_asyncio
from bson.objectid import ObjectId

client = motor.motor_asyncio.AsyncIOMotorClient(os.getenv("MONGODB_URI"))

In [3]:
project_name = "streamlit-prototype-projects"
db = client[f'{project_name}']

In [4]:
await client.list_database_names()

['streamlit-prototype-projects', 'admin', 'local']

In [5]:
collection = db['messages']

In [11]:
from pydantic import BaseModel
import datetime


class User(BaseModel):
    name: str
    email: str
    password: str
    courses: list[str]
    created_at: datetime.datetime
    updated_at: datetime.datetime


async def save_user(user: User):
    user_data = user.dict()
    await db['users'].insert_one(user_data)
    return user_data

async def get_user(user_id: str):
    return await db['users'].find_one({"_id": ObjectId(user_id)})

In [12]:
class AddMessage(BaseModel):
    role: str
    user_id: str
    message: str
    created_at: datetime.datetime
    updated_at: datetime.datetime

In [14]:
user = User.model_validate({"name": "Ayush", "email": "ayush@gmail.com", "password": "password",
                           "courses": ["1", "2"], "created_at": datetime.datetime.utcnow(), "updated_at": datetime.datetime.utcnow()})

In [10]:
save_user(user)

'Ayush'

In [8]:
async def save_user_message(role: str, user_id: str, message: str):
    if role not in ["system", "assistant", "user"]:
        raise ValueError("Invalid role. Role must be one of 'system', 'assistant', or 'user'.")

    message_data = {
        "role": role,
        "user_id": (user_id),
        "message": message,
        "created_at": datetime.datetime.utcnow(),
        "updated_at": datetime.datetime.utcnow()
    }

    await db['messages'].insert_one(message_data)
    return message_data

In [10]:
await save_user_message("user", "123", "Hello")

{'role': 'system',
 'user_id': '123',
 'message': 'Hello',
 'created_at': datetime.datetime(2024, 6, 27, 9, 42, 41, 408045),
 'updated_at': datetime.datetime(2024, 6, 27, 9, 42, 41, 408045),
 '_id': ObjectId('667d3411b57a98714d63d23d')}